In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.mam.*

In [ ]:
val model = Network("model")

// Create nodes
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val source = Source(model, "Source")
val sink = Sink(model, "Sink")

// Create job classes
val closedClass = ClosedClass(model, "ClosedClass", 2, delay, 0)
val openClass = OpenClass(model, "OpenClass", 0)

In [ ]:
// Set service time distributions
// Delay station serves both classes with different distributions
delay.setService(closedClass, Erlang(3.0, 2))  // Erlang distribution for closed class
delay.setService(openClass, HyperExp(0.5, 3.0, 10.0))  // Hyperexponential for open class

// Queue1 serves both classes
queue1.setService(closedClass, HyperExp(0.1, 1.0, 10.0))  // Hyperexponential for closed class
queue1.setService(openClass, Exp(1.0))  // Exponential for open class

// Set arrival process for open class
source.setArrival(openClass, Exp(0.1))

In [ ]:
// Set routing matrix
val P = model.initRoutingMatrix()

// Closed class routing: Delay <-> Queue1 (circular)
P.set(closedClass, closedClass, delay, queue1, 1.0)
P.set(closedClass, closedClass, queue1, delay, 1.0)

// Open class routing: Source -> Queue1 -> Sink
P.set(openClass, openClass, source, queue1, 1.0)
P.set(openClass, openClass, queue1, sink, 1.0)

model.link(P)

Mixed model created with:
- **Closed class**: 2 jobs circulating between Delay and Queue1
- **Open class**: Source → Queue1 → Sink
- **Different service distributions** for each class at each station

In [ ]:
// Default options matching MATLAB
val seed = 23000
val cutoff = 3
val verbose = true
val keep = true  // MATLAB has keep=true for this example
// val samples = 20000  // Commented in MATLAB as options.samples=2e4

// Create multiple solvers for comparison
val solvers = arrayOf(
    CTMC(model, "cutoff", cutoff),  // CTMC with cutoff
    JMT(model, "seed", seed, "verbose", verbose, "keep", keep),
    SSA(model, "seed", seed, "verbose", false, "cutoff", cutoff),  // Using same cutoff as MATLAB
    // FLD(model),  // Commented to match MATLAB
    MVA(model),
    // NC(model),  // Commented to match MATLAB
    MAM(model)
)

In [ ]:
// Solve and display results
for (solver in solvers) {
    try {
        val avgTable = solver.avgTable
        avgTable.print()
    } catch (e: Exception) {
        println("Error with ${solver.name}: ${e.message}")
    }
}

This mixed queueing network demonstrates several important concepts:

1. **Mixed Workloads**: Both closed and open classes share the same physical resources
2. **Resource Contention**: Queue1 serves both classes using processor sharing (PS)
3. **Different Service Requirements**: Each class has different service time distributions at each station
4. **Multiple Solution Methods**: Various solvers provide different accuracy/speed tradeoffs

**Key Observations**:
- The closed class circulates between Delay and Queue1 with 2 jobs
- The open class flows from Source through Queue1 to Sink
- Queue1 utilization is shared between both classes
- Different solvers may show slight variations due to approximation methods

**Solver Characteristics**:
- **CTMC**: Exact but computationally intensive
- **JMT**: Simulation-based, good for complex models
- **SSA**: Stochastic simulation, handles general distributions
- **Fluid**: Fast approximation for heavily loaded systems
- **MVA**: Efficient exact method for product-form networks
- **NC**: Normalizing constant method
- **MAM**: Matrix analytic methods for non-product-form features